In [ ]:
import pickle
import matplotlib.pyplot as plt
import adaptive_latents as al
import glob
import subprocess
import numpy as np
import time

In [ ]:
switch_iteration = 270
subprocess.call(["python", "run_partial_start.py", "gpu", str(switch_iteration)])
subprocess.call(["python", "run_partial_start.py", "cpu", str(switch_iteration)])

# these are to compare e.g. "cpu" vs "cpu cpu"
subprocess.call(["python", "run_partial_start.py", "gpu", "None"])
subprocess.call(["python", "run_partial_start.py", "cpu", "None"])

subprocess.call(["python", "run_partial_finish.py", "gpu", "2"])
subprocess.call(["python", "run_partial_finish.py", "cpu", "4"])

In [ ]:
files = sorted(glob.glob(f"{al.CONFIG["output_path"]/"bubblewrap_runs"}/*.pickle"))
brs = []
for file in files[-6:]:
    with open(file, 'br') as fhan:
        brs.append(pickle.load(fhan))
brs = np.array(brs)
brs = brs.reshape(-1,2)

In [ ]:
%matplotlib inline
plt.plot(brs[0,1].entropy_history[1] - brs[-1,1].entropy_history[1])
plt.axvline(switch_iteration-brs[0,0].bw.M, color='red')
print(switch_iteration - brs[0,0].bw.M)
print(np.nonzero(brs[0,1].entropy_history[1] - brs[-1,1].entropy_history[1] > 0)[0][0])

In [ ]:
%matplotlib inline
# red_lines=[brs[0,0].obs_ds.t[switch_iteration]]
al.plotting_functions.compare_metrics(brs.flatten(), 1, red_lines=[brs[0,0].obs_ds.t[switch_iteration-brs[0,0].bw.M]])

In [ ]:
def show_aspect_ratio_divergence(brs, bubble=5, switch=250):
    fig,ax = plt.subplots()
    
    m = bubble
    for k in range(len(brs)):
        ratios = []
        for i in range(brs[k].L_history.shape[0]):
            L = brs[k].L_history[i,m,...]
            _, s, _= np.linalg.svd(np.linalg.pinv(L.T@L))
            ratios.append(s.max() / s.min())
        ratios = np.log(ratios)
        ax.plot(ratios, label=brs[k].bw.backend_note)
    
    ax.set_title(f"log aspect ratio (bubble {m})");
    ax.legend();
    ax.axvline(switch-brs[0].bw.M, color='k', alpha=.25);
    ax.set_xlabel("iteration");
show_aspect_ratio_divergence(brs.flatten(), bubble=7, switch=switch_iteration)

## Inspect a single switch

In [ ]:
%matplotlib inline
br = brs[1][1]
dim = br.bw.mu.shape[1]

plt.plot((br.L_history[:,3,*np.tril_indices(dim, k=0)]))
plt.axvline(switch_iteration - brs[0,0].bw.M, color='k')
# plt.ylim([0,900])
plt.title(f"L for '{br.bw.backend_note}'");

In [ ]:
%matplotlib inline


plt.plot((br.L_lower_history[:,5,*np.tril_indices(dim, k=-1)]))
plt.axvline(switch_iteration - brs[0,0].bw.M, color='k')
plt.title(f"L_lower for '{br.bw.backend_note}'");

In [ ]:
%matplotlib inline
dim = br.bw.mu.shape[1]

bubble=4
plt.plot((br.L_diag_history[:,bubble,...]))
plt.axvline(switch_iteration - brs[0,0].bw.M, color='k')
plt.title(f"L_diag for '{br.bw.backend_note}'");

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(nrows=3, layout="tight", figsize=(10,5), sharex=True)

bubble = 5
axs[0].plot(br.L_diag_grad_history[:, bubble, :])
axs[0].set_ylim(np.array([-1,1])*200)

axs[1].plot(br.L_diag_v_history[:, bubble, :])
axs[1].set_ylim(np.array([-1,1])*.01 + 0.005)


axs[2].plot(br.L_diag_m_history[:, bubble, :])
axs[2].set_ylim(np.array([-1,1])*1)


for i in range(3):
    axs[i].axvline(switch_iteration - brs[0,0].bw.M, color='k')


In [ ]:
%matplotlib qt
fig, axs = plt.subplots(nrows=3, layout="tight", figsize=(10,5), sharex=True)

bubble = 5
axs[0].plot(br.L_diag_grad_history[:, bubble, :])
axs[0].set_ylim(np.array([-1,1])*200)

axs[1].plot(br.L_diag_v_history[:, bubble, :])
axs[1].set_ylim(np.array([-1,1])*.01 + 0.005)


axs[2].plot(br.L_diag_m_history[:, bubble, :])
axs[2].set_ylim(np.array([-1,1])*1)


for i in range(3):
    axs[i].axvline(switch_iteration - brs[0,0].bw.M, color='k')


In [ ]:
br.L_diag_grad_history[switch_iteration - brs[0,0].bw.M -10, bubble, :]

In [ ]:
L = br.L_history[-1][0]
L = np.linalg.inv(L)
L @ L.T

In [ ]:
np.tril(np.diag(np.exp(br.bw.L_lower[0]) + 1e-10) + np.tril(br.bw.L_diag[0], -1))

In [ ]:
# 
L = np.tril(np.diag(np.exp(br.bw.L_lower[0]) + 1e-10) + np.tril(br.bw.L_diag[0], -1))
el = np.linalg.inv(L)
el.T @ el

In [ ]:
br.L_history[-1,5,...]

In [ ]:
%matplotlib inline
plt.plot(np.log(np.abs(br.L_grad_history[:,5,...].reshape(-1,br.bw.mu.shape[1]**2))))
# plt.ylim([-.01,.01])
plt.axvline(switch_iteration - brs[0,0].bw.M, color='k')
